# Running nustarpipeline to Produce Science Products for NuSTAR:

Adopted from C. Ferrigno (ISDC, University of Geneva), https://gitlab.astro.unige.ch/ferrigno/nustar-pipeline

In [ ]:
## Importing modules developed for this project:
# eg nustarpipeline process to wrap the nustar analysis for our needs, 
# nustarpipeline utils to collect useful functions
# pyxmmsas (originally for XMM) for spectral fitting

%matplotlib widget
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.stats import norm
import shutil, os, sys, hratio, json, logging
from astroquery.simbad import Simbad
from astropy import units as u
from astropy.coordinates import SkyCoord
from importlib import reload
from astropy.io import fits as pf
from nustarpipeline import process, utils
import pyxmmsas as pysas
from scipy import interpolate
from datetime import datetime 
from time import gmtime, strftime

In [ ]:
#Set your source name and output folder:
source : str = 'src' 
output_dir_figures_name : str = 'figures' 
orbitfile : str = None #'orbit.dat' # None to avoid orbit correction
    
#Set "False" to speed up reprocessing, use "True" to produce the lightcurves and spectra for FPMA and FPMB:
run_nustar_tasks :bool = True 
rerun_nustar_pipeline :bool = True 

In [ ]:
#Set your working directory and ObsID:
mylocation= os.getcwd()
print('Working directory = ' + mylocation)
obsid='30202022002'
print('OBSID: '+ obsid)
if str(orbitfile) == 'None':
    orbitfile = None

#Set local pfiles for HEASOFT:

if os.path.isdir('pfiles'):
    print('folder \'pfiles\' exist')
else:
    os.makedirs('pfiles')
pfiles = os.environ['PFILES'].split(';')
os.environ['PFILES']=mylocation+'/pfiles;'+pfiles[1]
print('PFILES:' + os.environ['PFILES'])

#Set the output folder
if os.path.isdir(output_dir_figures_name):
    print('folder \'%s\' exist' % output_dir_figures_name)
else:
    os.makedirs(output_dir_figures_name)
    
output_dir_figures = mylocation+'/'+output_dir_figures_name

file_handler = logging.FileHandler(filename='nustar_utils_%s.log' % (strftime("%Y-%m-%dT%H:%M:%S", gmtime())))
stdout_handler = logging.StreamHandler(sys.stdout)
handlers = [file_handler] #stdout_handler, 

logging.basicConfig(level=logging.INFO, format=' %(levelname)s - %(message)s', handlers=handlers)
logging.getLogger('').addHandler(logging.StreamHandler()) 

reload(utils)

tmp = mylocation.split('/')

obsid='30202022002'
ra_src,dec_src = utils.get_target_coords_extern(source)
print(obsid)

In [ ]:
reload(process)
if rerun_nustar_pipeline:
    process.wrap_process(obsid, ra_src, dec_src, obsid, pipeline_flag=True, 
                            repository_location='/Volumes/Ana_PhD/data/ULXs/M82/NuSTAR/')

Do not forget to define your regions for FPMA and FPMB as:

- FPMA: sourceA.reg & backgroundA.reg
- FPMB: sourceA.reg & backgroundB.reg


In [ ]:
if run_nustar_tasks:
    reload(process)

    process.wrap_process(obsid, ra_src, dec_src, obsid, region_flag=True,  no_ds9_flag=True,
                     repository_location='/Volumes/Ana_PhD/data/ULXs/M82/NuSTAR/')
 